                                                                            Luis Ramirez Camargo, June 2020

# Clean up PV output data of the Chilean installations
This notebook generates CSV files with the merged and clean time series of PV electricity generation for each large PV installation in Chile.
See related notebooks: 2_get_time_series_from_era5_land_and_merra2_for_pv_calculation, 3_pv_output_from_ERA5_land_and_merra2 and 4_pv_validation_ERA5_land_MERRA2


In [1]:
import os
import itertools
import xarray as xr
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import geopandas
import glob
from tkinter import Tcl
import gc
import unidecode
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import scipy as sp
import scipy.stats
from pathlib import Path
import numpy as np

## Import locations

In [2]:
installations_chile_pre = \
Path("input_data/solares_fd0779de_0870_4194_b962_83a842d8c316.shp")
def clean_installations_locations(installations_chile_pre):
    '''creates a data frame with the location and basic 
    characteristics of the large PV installations in Chile'''
    pvs_chile = geopandas.read_file(installations_chile_pre)
    pvs_chile_4326 = pvs_chile.to_crs(epsg=4326)
    installations_list = pvs_chile_4326["NOMBRE"]
    installations_chile = pd.DataFrame({'latitude': pvs_chile_4326.geometry.y.values, 
                        'longitude': pvs_chile_4326.geometry.x.values, 
                        'size': pvs_chile_4326['POTENCIA'].values, 
                        'official_operation_start': pd.to_datetime(pvs_chile_4326['F_OPERACIO'].values), 
                        'end_time': pd.to_datetime('2018-12-30')},
                       index=installations_list)
    return installations_chile

installations_chile = clean_installations_locations(installations_chile_pre)

## Import time series and match with location
The only link between the location data and the time series is the name of the installation. These were in most of the cases writen in different ways. The following section is a semi automatic matching of the data in both data sets. The list "installations_with_no_match" and the dictionary "forced_match" were manually constructed after several iterations of the matching attempts with the fuzzy string matching function. Only the final runing version is provide here and is can be seen they are interdependent. That is why the list and the dictionary are defined before the function.

In [3]:
#load the list of files with pv generation data from energía abierta for the sen 
#(should include sic and sing 
#https://www.coordinador.cl/operacion/graficos/operacion-real/generacion-real-del-sistema/?radio-formato=xlsx)
time_series_files = Path("input_data/energia_abierta/*.xlsx")
def get_list_of_pv_files(time_series_files):
    generation_chile_pre_sen = list(glob.glob(str(time_series_files)))
    generation_chile_sen = Tcl().call('lsort', '-dict', generation_chile_pre_sen)
    return generation_chile_sen

In [4]:
generation_chile_sen = get_list_of_pv_files(time_series_files)

In [5]:
#define a list of installations names that do not have a match
installations_with_no_match = ("SOLAR PSF PAMA",  
                               "SOLAR EL PILAR - LOS AMARILLOS", 
                               "SOLAR LOMA LOS COLORADOS",
                               "SOLAR LUNA",
                               "SOLAR DONA CARMEN",
                               "SOLAR EL PELICANO", 
                               "SOLAR PAMA", 
                               "SOLAR OLIVILLO",
                               "SOLAR EL CHINCOL",
                               "SOLAR CATAN", 
                               "SOLAR LAS PALOMAS")
#define a list of installations, which automatic matching using 
#fuzzy matching might lead to multiple installations 
#being associated to on single time series
forced_match = {"SOLAR LAGUNILLA":"PFV LAGUNILLA", 
                "SOLAR LALACKAMA 2":"LALACKAMA 2", 
                "SOLAR LALACKAMA":"LALACKAMA",
                "SOLAR TAMBO REAL":"TAMBO REAL",
                "SOLAR SAN ANDRÉS":"SOLAR SAN ANDRES",
                "SOLAR LUZ DEL NORTE":"LUZ DEL NORTE",
                "SOLAR PAMPA SOLAR NORTE":"PAMPA SOLAR NORTE",
                "SOLAR PV CONEJO":"CONEJO SOLAR", 
                "SOLAR CARRERA PINTO":"CARRERA PINTO ETAPA I",
                "SOLAR SAN PEDRO":"PFV SAN PEDRO",
                "SOLAR LA QUINTA":"LA QUINTA SOLAR",
                "SOLAR CERNICALO 1":"EL CERNICALO 1",
                "SOLAR CERNICALO 2":"EL CERNICALO 2",
                "SOLAR SAN FRANCISCO":"SAN FRANCISCO SOLAR",
                "SOLAR TALHUEN":"TALHUEN", 
                "SOLAR ANDES":"ANDES SOLAR", 
                "SOLAR LA HUAYCA 2":"SPS LA HUAYCA", 
                "SOLAR PILOTO CARDONES":"PILOTO SOLAR CARDONES",
                "SOLAR LUDERS":"PFV LUDERS", 
                "SOLAR TIL TIL":"TILTIL SOLAR", 
                "SOLAR PUERTO SECO":"PUERTO SECO SOLAR", 
                "SOLAR SANTIAGO":"SANTIAGO SOLAR", 
                "SOLAR URIBE":"URIBE SOLAR",
                "SOLAR SAN FRANCISCO ":"SAN FRANCISCO SOLAR",
                "SOLAR PICA ":"PMGD PICA PILOT",
                "SOLAR SOL":"SOL DEL NORTE", 
                "SOLAR OCOA":"OCOA", 
                "SOLAR PORTEZUELO":"PORTEZUELO", 
                "SOLAR SAN FRANCISCO":"SAN FRANCISCO SOLAR",
                "SOLAR VILLA PRAT":"VILLA PRAT",
                "SOLAR CALAMA 1":"CALAMA SOLAR 1",
                "SOLAR FRANCISCO":"FRANCISCO SOLAR", 
                "SOLAR RODEO":"RODEO",
                "SOLAR SANTA LAURA":"SANTA LAURA", 
                "SOLAR SANTUARIO":"SANTUARIO SOLAR"}

In [6]:
def installations_matching(installations_list,matching_ratio):
    '''creates a data frame of the time series of electricity ouput using the names of the PV installations
    that are originally comming from the data set with the locations'''
    date_range = pd.date_range(start='1/1/2014', end='01/01/2019', freq='H', tz='Chile/Continental')
    date_range_m = pd.date_range(start='1/1/2014', end='31/12/2018', freq='M', tz='Chile/Continental')
    days_all = pd.date_range(start='1/1/2014', end='31/12/2018', freq='D')
    timeseries_installations_sen = pd.DataFrame(columns=installations_list, index=pd.to_datetime(date_range))
    file = 0
    day_counter = 0
    for month in date_range_m:
        output_chile = pd.read_excel(generation_chile_sen[file], sheet_name='Sheet', header=3)
        for day in days_all[day_counter:]:
            #print(day)
            output_chile_day = (output_chile.loc[(output_chile["Tipo"]=="Solar") &
                                             (output_chile["Fecha"]==((str(day))[:-9]))])
            start_hour = day_counter * 24
            stop_hour = start_hour + 24
            #print(str(day_file)+"  "+str(start_hour)+" "+str(stop_hour))
            #go through all installations in a day and store the hourly values in timeseries_installations_sen
            for installations in output_chile_day.index:
                installation = unidecode.unidecode(output_chile_day.loc[installations]["Grupo reporte"].upper())
                #print(installation)
                hourly_values = output_chile_day.loc[installations][6:30]
                if installation in timeseries_installations_sen.columns:
                    timeseries_installations_sen[installation][start_hour:stop_hour] = \
                    hourly_values
                    #print(str(installation) + "*")
                elif str('SOLAR '+str(installation)) in timeseries_installations_sen.columns:
                    timeseries_installations_sen['SOLAR '+str(installation)][start_hour:stop_hour] = \
                    hourly_values
                    #print(str(installation) + "**")
                elif installation in installations_with_no_match:
                    #print(str(installation) + " is part of the list without a match")
                    pass
                elif installation in forced_match:
                    #print(str(installation) + " is part of the list with forced match")
                    timeseries_installations_sen[forced_match[installation]][start_hour:stop_hour] = \
                    hourly_values
                else:
                    #print(installation)
                    running_ratio = matching_ratio
                    for installation_not_match in timeseries_installations_sen.columns:
                        if fuzz.partial_ratio(installation,installation_not_match)> matching_ratio and \
                        fuzz.partial_ratio(installation,installation_not_match)> running_ratio:
                            running_ratio = fuzz.partial_ratio(installation,installation_not_match)
                            #print(str(fuzz.partial_ratio(installation,installation_not_match)) + " " + 
                                    #str(installation)+ " " +str(installation_not_match))
                            timeseries_installations_sen[installation_not_match][start_hour:stop_hour] = \
                            hourly_values
            day_counter += 1
            if day in date_range_m:
                file += 1
                #print(str(day)+" end of the month "+str(file))
                break
    return timeseries_installations_sen

In [7]:
timeseries_installations_sen = installations_matching(installations_chile.index,70)

In [8]:
#make a copy of the raw data 
#timeseries_installations_sen.to_csv('intermediate_results/time_series_PV_sen_chile_raw.csv')

# clean the data for outlayers and calculate capacity factors

In [9]:
def calculate_cf_meassured_data(timeseries_installations_sen):
    '''creates a df of capacity factor values for the meassured
    data from the PV installations after cleaning up for outlayers'''
    reference_pre1 = timeseries_installations_sen
    for i in timeseries_installations_sen.columns:
        generation_raw1 = timeseries_installations_sen[i].dropna().copy()
        if np.sum(generation_raw1) > 0:
            installation_size = (np.percentile(generation_raw1,99))
            outliers_threshold = installation_size * 1.1
            #transfrom the outlayers into nan
            generation_raw2 = timeseries_installations_sen[i].where(timeseries_installations_sen[i] < outliers_threshold, 
                                                                    np.nan).copy()
            #normalize by the maximum value after cleaning the outlayers
            #reference_pre1[i] = ((generation_raw2/np.max(generation_raw2))*1000).copy()
            reference_pre1[i] = ((generation_raw2/np.max(generation_raw2))).copy()
            #uncomment to se examples of the outlayers cleaning
            #print(i)
            #print(installation_size)
            #print(outliers_threshold)
            #fig = plt.figure(figsize=(20,6))
            #plt.plot(generation_raw1)
            #plt.plot(generation_raw2)
            #plt.show()               

    ###all 0 values are transformed to non data 
    reference = reference_pre1.tz_convert('UTC').where(reference_pre1 > 0.0, np.nan).copy()
    return reference

In [10]:
reference = calculate_cf_meassured_data(timeseries_installations_sen)

In [11]:
reference.to_csv(Path('intermediate_results/time_series_PV_sen_chile_capacity_factors.csv'))